In [0]:
import pandas as pd
import numpy as np
import math

In [2]:
from google.colab import drive
drive.mount("/gdrive")

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
#loading clean preprocessed data(done in prevoius excercises)
colleges = pd.read_csv("/gdrive/My Drive/colleges_cleaned.csv")
colleges.describe()

,Unnamed: 0,Overall Rank,Undergrad. Enrollment,Admission Rate,Student/faculty Ratio,4-year Grad. Rate,6-year Grad. Rate,Quality Rank,Total Costs,Cost After Need-based Aid,Need Met,Aid From Grants,Cost After Non-Need-Based Aid,Non-Need-Based Aid+,Average Debt,Cost Rank
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,56.027778,56.638889,3771.375000,0.479722,0.495370,0.693194,0.803889,59.319444,0.801604,0.580527,0.924583,0.746806,0.599351,0.322639,0.543734,43.041667
std,27.366563,27.204652,4503.418768,0.176985,0.160915,0.160911,0.068558,25.087917,0.198884,0.173434,0.139172,0.112923,0.196497,0.252056,0.227119,28.238141
min,0.000000,1.000000,67.000000,0.140000,0.000000,0.000000,0.670000,4.000000,0.000000,0.000000,0.200000,0.270000,0.000000,0.010000,0.000000,1.000000
25%,34.750000,35.750000,1625.500000,0.340000,0.400000,0.630000,0.750000,40.750000,0.708513,0.504470,0.907500,0.700000,0.487612,0.110000,0.492894,18.750000
50%,59.500000,59.500000,2279.000000,0.445000,0.533333,0.720000,0.810000,60.500000,0.873636,0.581522,0.985000,0.760000,0.595649,0.255000,0.555676,39.500000
75%,77.250000,77.250000,4257.750000,0.627500,0.550000,0.790000,0.852500,79.250000,0.944257,0.672169,1.000000,0.802500,0.716354,0.485000,0.669924,69.250000
max,99.000000,100.000000,29379.000000,0.870000,1.000000,0.890000,0.950000,100.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,100.000000


In [4]:
colleges.columns

Index(['Unnamed: 0', 'Overall Rank', 'School', 'State',
       'Undergrad. Enrollment', 'Admission Rate', '*SAT or ACT',
       'Student/faculty Ratio', '4-year Grad. Rate', '6-year Grad. Rate',
       'Quality Rank', 'Total Costs', 'Cost After Need-based Aid', 'Need Met',
       'Aid From Grants', 'Cost After Non-Need-Based Aid',
       'Non-Need-Based Aid+', 'Average Debt', 'Cost Rank'],
      dtype='object')

In [0]:
def get_transactions(attributes):
    transactions = []
    for i in colleges.index:
        item_set = set()
        for attribute in attributes:
            item_set.add(int(10*colleges.loc[i,attribute]))
        transactions.append(item_set)
    return transactions

In [0]:
attributes = ['6-year Grad. Rate','Total Costs','Cost After Non-Need-Based Aid',
              'Non-Need-Based Aid+','Admission Rate']
transactions = get_transactions(attributes)

In [7]:
print(*transactions[:10],sep='\n')

{1, 2, 3, 7, 8}
{2, 3, 5, 6, 8}
{0, 9, 2, 1}
{8, 0, 10, 4}
{8, 3, 4, 6}
{1, 4, 5, 8, 9}
{0, 9, 2, 6}
{8, 9, 3, 7}
{9, 3, 1}
{0, 9, 10, 2}


In [0]:
def create_candidates(dataset, VERBOSE=False):
    """Creates a list of candidate 1-itemsets from a list of transactions.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    Returns
    -------
    The list of candidate itemsets (c1) passed as a frozenset (a set that is 
    immutable and hashable).
    """
    c1 = [] # list of all items in the database of transactions
    for transaction in dataset:
        for item in transaction:
            if not [item] in c1:
                c1.append([item])
    c1.sort()

    if VERBOSE:
        # Print a list of all the candidate items.
        print ("" \
            + "{" \
            + "".join(str(i[0]) + ", " for i in iter(c1)).rstrip(', ') \
            + "}")

    # Map c1 to a frozenset because it will be the key of a dictionary.
    return list(map(frozenset, c1))

In [9]:
#generate candidate itemsets
C1 = create_candidates(transactions,VERBOSE=True)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}


In [0]:
def support_prune(dataset, candidates, min_support, VERBOSE=False):
    """Returns all candidate itemsets that meet a minimum support threshold.

    By the apriori principle, if an itemset is frequent, then all of its 
    subsets must also be frequent. As a result, we can perform support-based 
    pruning to systematically control the exponential growth of candidate 
    itemsets. Thus, itemsets that do not meet the minimum support level are 
    pruned from the input list of itemsets (dataset).

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate 
        itemsets.

    candidates : frozenset
        The list of candidate itemsets.

    min_support : float
        The minimum support threshold.

    Returns
    -------
    retlist : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.
    """
    sscnt = {} # set for support counts
    num_items=0
    for tid in dataset:
        num_items+=1
        for candidate in candidates:
            if candidate.issubset(tid):
                sscnt.setdefault(candidate, 0)
                sscnt[candidate] += 1

    #num_items = 5 #len(dataset) # total number of transactions in the dataset
    retlist = [] # array for unpruned itemsets
    support_data = {} # set for support data for corresponding itemsets
    for key in sscnt:
        # Calculate the support of itemset key.
        support = sscnt[key] / num_items
        if support >= min_support:
            retlist.insert(0, key)
        support_data[key] = support

    # Print a list of the pruned itemsets.
    if VERBOSE:
        for kset in retlist:
            for item in kset:
                print ("{" + str(item) + "}")
        #print
        for key in sscnt:
            print ("" \
                + "{" \
                + "".join([str(i) + ", " for i in iter(key)]).rstrip(', ') \
                + "}" \
                + ":  sup = " + str(support_data[key]))

    return retlist, support_data

In [11]:
F1, support_data = support_prune(transactions, C1, 0.5, VERBOSE=True)


{8}
{7}
{1}:  sup = 0.2361111111111111
{2}:  sup = 0.2638888888888889
{3}:  sup = 0.3611111111111111
{7}:  sup = 0.5833333333333334
{8}:  sup = 0.6111111111111112
{5}:  sup = 0.3888888888888889
{6}:  sup = 0.4166666666666667
{0}:  sup = 0.2222222222222222
{9}:  sup = 0.4166666666666667
{4}:  sup = 0.4444444444444444
{10}:  sup = 0.05555555555555555


In [0]:
def apriori(dataset, min_support=0.5, VERBOSE=False):
    """Implements the Apriori algorithm.

    The Apriori algorithm will iteratively generate new candidate 
    k-itemsets using the frequent (k-1)-itemsets found in the previous 
    iteration.

    Parameters
    ----------
    dataset : list
        The dataset (a list of transactions) from which to generate candidate itemsets.

    min_support : float
        The minimum support threshold. Defaults to 0.5.

    Returns
    -------
    F : list
        The list of frequent itemsets.

    support_data : dict
        The support data for all candidate itemsets.


    """
    C1 = create_candidates(dataset)
    D = map(set, dataset)
    F1, support_data = support_prune(dataset, C1, min_support, VERBOSE=False) # prune candidate 1-itemsets
    print("f = ",F1)
    F = [F1] # list of frequent itemsets; initialized to frequent 1-itemsets
    k = 2 # the itemset cardinality
    while (len(F[k - 2]) > 0):
        Ck = apriori_gen(F[k-2], k) # generate candidate itemsets
        #print("\nCk",Ck)
        Fk, supK = support_prune(dataset, Ck, min_support) # prune candidate itemsets
        #print("f = ",F)
        support_data.update(supK) # update the support counts to reflect pruning
        F.append(Fk) # add the pruned candidate itemsets to the list of frequent itemsets
        k += 1

    
    return F, support_data

def apriori_gen(freq_sets, k):
    """Generates candidate itemsets (via the F_k-1 x F_k-1 method).

    This operation generates new candidate k-itemsets based on the frequent 
    (k-1)-itemsets found in the previous iteration. The candidate generation 
    procedure merges a pair of frequent (k-1)-itemsets only if their first k-2 
    items are identical.

    Parameters
    ----------
    freq_sets : list
        The list of frequent (k-1)-itemsets.

    k : integer
        The cardinality of the current itemsets begin evaluated.

    Returns
    -------
    retlist : list
        The list of merged frequent itemsets.
    """
    retList = [] # list of merged frequent itemsets
    lenLk = len(freq_sets) # number of frequent itemsets
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            a=list(freq_sets[i])
            b=list(freq_sets[j])
            a.sort()
            b.sort()
            F1 = a[:k-2] # first k-2 items of freq_sets[i]
            F2 = b[:k-2] # first k-2 items of freq_sets[j]

            if F1 == F2: # if the first k-2 items are identical
                # Merge the frequent itemsets.
                retList.append(freq_sets[i] | freq_sets[j])

    return list(retList)


In [13]:
# Generate all the frequent itemsets using the Apriori algorithm.
F, support_data = apriori(transactions, min_support=0.15, VERBOSE=True)

f =  [frozenset({4}), frozenset({9}), frozenset({0}), frozenset({6}), frozenset({5}), frozenset({8}), frozenset({7}), frozenset({3}), frozenset({2}), frozenset({1})]


In [14]:
print(support_data)

{frozenset({1}): 0.2361111111111111, frozenset({2}): 0.2638888888888889, frozenset({3}): 0.3611111111111111, frozenset({7}): 0.5833333333333334, frozenset({8}): 0.6111111111111112, frozenset({5}): 0.3888888888888889, frozenset({6}): 0.4166666666666667, frozenset({0}): 0.2222222222222222, frozenset({9}): 0.4166666666666667, frozenset({4}): 0.4444444444444444, frozenset({10}): 0.05555555555555555, frozenset({8, 7}): 0.25, frozenset({8, 3}): 0.2638888888888889, frozenset({8, 2}): 0.125, frozenset({8, 1}): 0.1527777777777778, frozenset({3, 7}): 0.1527777777777778, frozenset({2, 7}): 0.16666666666666666, frozenset({1, 7}): 0.1111111111111111, frozenset({2, 3}): 0.06944444444444445, frozenset({1, 3}): 0.06944444444444445, frozenset({1, 2}): 0.05555555555555555, frozenset({5, 6}): 0.19444444444444445, frozenset({8, 6}): 0.16666666666666666, frozenset({3, 6}): 0.1111111111111111, frozenset({2, 6}): 0.09722222222222222, frozenset({8, 5}): 0.2638888888888889, frozenset({3, 5}): 0.111111111111111

In [0]:
def rules_from_conseq(freq_set, H, support_data, rules, min_confidence=0.7):
    """Generates a set of candidate rules.

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A list of frequent itemsets (of a particular length).

    support_data : dict
        The support data for all candidate itemsets.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.
    """
    m = len(H[0])
    if (len(freq_set) > (m+1)):
        Hmp1 = apriori_gen(H, m+1) # generate candidate itemsets
        Hmp1 = calc_confidence(freq_set, Hmp1,  support_data, rules, min_confidence)
        if len(Hmp1) > 1:
            # If there are candidate rules above the minimum confidence 
            # threshold, recurse on the list of these candidate rules.
            rules_from_conseq(freq_set, Hmp1, support_data, rules, min_confidence)

def calc_confidence(freq_set, H, support_data, rules, min_confidence=0.7, VERBOSE=False):
    """Evaluates the generated rules.

    One measurement for quantifying the goodness of association rules is 
    confidence. The confidence for a rule 'P implies H' (P -> H) is defined as 
    the support for P and H divided by the support for P 
    (support (P|H) / support(P)), where the | symbol denotes the set union 
    (thus P|H means all the items in set P or in set H).

    To calculate the confidence, we iterate through the frequent itemsets and 
    associated support data. For each frequent itemset, we divide the support 
    of the itemset by the support of the antecedent (left-hand-side of the 
    rule).

    Parameters
    ----------
    freq_set : frozenset
        The complete list of frequent itemsets.

    H : list
        A frequent itemset.

    min_support : float
        The minimum support threshold.

    rules : list
        A potentially incomplete set of candidate rules above the minimum 
        confidence threshold.

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    pruned_H : list
        The list of candidate rules above the minimum confidence threshold.
    """
    pruned_H = [] # list of candidate rules above the minimum confidence threshold
    for conseq in H: # iterate over the frequent itemsets
        conf = support_data[freq_set] / support_data[freq_set - conseq]
        if conf >= min_confidence:

            rules.append((freq_set - conseq, conseq, conf))
            pruned_H.append(conseq)

            if VERBOSE:
                print( "" \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(freq_set-conseq)]).rstrip(', ') \
                    + "}" \
                    + " ---> " \
                    + "{" \
                    + "".join([str(i) + ", " for i in iter(conseq)]).rstrip(', ') \
                    + "}" \
                    + ":  conf = " + str(round(conf, 3)) \
                    + ", sup = " + str(round(support_data[freq_set], 3)))

    return pruned_H

def generate_rules(F, support_data, min_confidence, VERBOSE=False):
    """Generates a set of candidate rules from a list of frequent itemsets.

    For each frequent itemset, we calculate the confidence of using a
    particular item as the rule consequent (right-hand-side of the rule). By 
    testing and merging the remaining rules, we recursively create a list of 
    pruned rules.

    Parameters
    ----------
    F : list
        A list of frequent itemsets.

    support_data : dict
        The corresponding support data for the frequent itemsets (L).

    min_confidence : float
        The minimum confidence threshold. Defaults to 0.7.

    Returns
    -------
    rules : list
        The list of candidate rules above the minimum confidence threshold.
    """
    rules = []
    for i in range(1, len(F)):
        for freq_set in F[i]:
            H1 = [frozenset([item]) for item in freq_set]

            if (i > 1):
                rules_from_conseq(freq_set, H1, support_data, rules, min_confidence)
            else:
                calc_confidence(freq_set, H1, support_data, rules, min_confidence, VERBOSE=VERBOSE)

    return rules

In [16]:
# Generate the association rules from a list of frequent itemsets.
H = generate_rules(F, support_data, min_confidence=0.01, VERBOSE=True)

{7} ---> {6}:  conf = 0.5, sup = 0.292
{6} ---> {7}:  conf = 0.7, sup = 0.292
{7} ---> {5}:  conf = 0.333, sup = 0.194
{5} ---> {7}:  conf = 0.5, sup = 0.194
{7} ---> {4}:  conf = 0.524, sup = 0.306
{4} ---> {7}:  conf = 0.688, sup = 0.306
{3} ---> {9}:  conf = 0.5, sup = 0.181
{9} ---> {3}:  conf = 0.433, sup = 0.181
{7} ---> {9}:  conf = 0.333, sup = 0.194
{9} ---> {7}:  conf = 0.467, sup = 0.194
{9} ---> {8}:  conf = 0.6, sup = 0.25
{8} ---> {9}:  conf = 0.409, sup = 0.25
{5} ---> {4}:  conf = 0.429, sup = 0.167
{4} ---> {5}:  conf = 0.375, sup = 0.167
{6} ---> {4}:  conf = 0.533, sup = 0.222
{4} ---> {6}:  conf = 0.5, sup = 0.222
{8} ---> {0}:  conf = 0.25, sup = 0.153
{0} ---> {8}:  conf = 0.688, sup = 0.153
{4} ---> {8}:  conf = 0.625, sup = 0.278
{8} ---> {4}:  conf = 0.455, sup = 0.278
{2} ---> {9}:  conf = 0.579, sup = 0.153
{9} ---> {2}:  conf = 0.367, sup = 0.153
{9} ---> {0}:  conf = 0.367, sup = 0.153
{0} ---> {9}:  conf = 0.688, sup = 0.153
{5} ---> {8}:  conf = 0.679, su

In [19]:
H = sorted(H, key=lambda x:x[2], reverse = True)
print(*H[:10],sep='\n')

(frozenset({3}), frozenset({8}), 0.7307692307692308)
(frozenset({6}), frozenset({7}), 0.7000000000000001)
(frozenset({4}), frozenset({7}), 0.6875000000000001)
(frozenset({0}), frozenset({8}), 0.6875000000000001)
(frozenset({0}), frozenset({9}), 0.6875000000000001)
(frozenset({5}), frozenset({8}), 0.6785714285714286)
(frozenset({1}), frozenset({8}), 0.6470588235294118)
(frozenset({2}), frozenset({7}), 0.631578947368421)
(frozenset({4}), frozenset({8}), 0.6250000000000001)
(frozenset({9}), frozenset({8}), 0.6)
